# Google Colab Settings

In [1]:
!pip install catboost

     |████████████████████████████████| 66.3MB 68kB/s 


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Release note

全新套路，https://github.com/jiahengqi/datacastle_shixin/blob/master/runner.py

In [1]:
import pickle,os,warnings
import pandas as pd
import numpy as np
from collections import Counter
from tqdm import trange
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from sklearn.metrics import roc_auc_score
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
warnings.filterwarnings('ignore')

In [2]:
# import os
# os.chdir('/content/drive/My Drive/Colab Notebooks/MachineLearningPractice/GongJiJin')

In [3]:
# def score(data,model):
#     y_pred=model.predict_proba(data[feat])[:, 1]
#     return roc_auc_score(data.Label, y_pred)

In [90]:
# data_train = pd.read_csv('originalDataset/train.csv')
# data_test_a = pd.read_csv('originalDataset/test.csv')

train = pd.read_csv('originalDataset/train.csv')
train_y = train.label
test = pd.read_csv('originalDataset/test.csv')

In [91]:
featuresToDiscard = ["HYZK", "ZHIYE", "ZHICHEN", "ZHIWU", "XUELI"]
train.drop(featuresToDiscard, axis = 1, inplace = True)
test.drop(featuresToDiscard, axis = 1, inplace = True)

In [92]:
## 过滤类别较少的和较多的列
def get_numerical_serial_fea(data,feas):
    numerical_serial_fea = []
    numerical_noserial_fea = []
    for fea in feas:
        temp = data[fea].nunique()
        if temp <= 40:
            numerical_noserial_fea.append(fea)
            continue
        numerical_serial_fea.append(fea)
    return numerical_serial_fea,numerical_noserial_fea
    
serial_fea_, categorical_fea_ = get_numerical_serial_fea(train,train.columns)
categorical_fea = list(filter(lambda x: x not in ["DKLL", "label"], categorical_fea_))
serial_fea = list(filter(lambda x: x not in ["id", "CSNY"], serial_fea_))
serial_fea.append("DKLL")
print("serial features: ", serial_fea)
print("categorical features: ", categorical_fea)

serial features:  ['GRJCJS', 'GRZHYE', 'GRZHSNJZYE', 'GRZHDNGJYE', 'GRYJCE', 'DWYJCE', 'DKFFE', 'DKYE', 'DKLL']
categorical features:  ['XINGBIE', 'DWJJLX', 'DWSSHY', 'GRZHZT']


In [93]:
cat_feat = categorical_fea #['登记机关', '行业代码', '行业门类', '企业类型']
feat = serial_fea #list(set(train.columns)-set(train.select_dtypes(object))-set(['Label', 'ID'])-set(cat_feat))
remove_col = []
for col in feat:
    if train[col].nunique() < 2:
        remove_col.append(col)
feat = list(set(feat) - set(remove_col))

In [94]:
cat_feat

['XINGBIE', 'DWJJLX', 'DWSSHY', 'GRZHZT']

In [95]:
feat

['DKLL',
 'GRJCJS',
 'DKFFE',
 'GRZHDNGJYE',
 'GRZHYE',
 'DKYE',
 'DWYJCE',
 'GRZHSNJZYE',
 'GRYJCE']

In [96]:
class Feat:
    def __init__(self, config):
        self.config = config
    
    def fit(self, x, y):
        pass

    def transform(self, x):
        pass
    
    def fit_transform(self, x, y):
        self.fit(x, y)
        self.transform(x)
        
class CatCount(Feat):
    def transform(self, x):
        for col in self.config['cat_columns']:
            df_count = x[col].value_counts()
            x[f'{col}_catcount'] = x[col].map(df_count)

class CatCountRank(Feat):
    def fit(self, x, y):
        self.fit_dict = {}
        for col in self.config['cat_columns']:
            counter =  Counter(x[col]).most_common()
            self.fit_dict[col] = {k: i for (i, (k, v)) in enumerate(counter)}

    def transform(self, x):
        for col in self.config['cat_columns']:
            x[f'{col}_countrank'] = x[col].map(self.fit_dict[col])

In [97]:
config = {}
config['cat_columns'] = cat_feat

In [98]:
categorical_fea_index = []
for i, col in enumerate(train.columns):
    if col in cat_feat:
        categorical_fea_index.append(i)

params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'min_child_weight': 5,
    'num_leaves': 2 ** 5,
    'lambda_l2': 10,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 4,
    'learning_rate': 0.1,
    'seed': 2020,
    'nthread': 28,
    'n_jobs':24,
    'verbose': -1,

    ## 
    "categorical_feature": categorical_fea_index, #"name:{}".format(",".join(categorical_fea)),

    #########
#                 'silent': True,
    # 'metric': 'auc',
}

比较重要的数据处理步骤来了哦，注意，别盲目跑代码cell了。

In [ ]:
feat0 = feat.copy()
for i in trange(len(feat)):
    df_temp=train[feat0].copy()
    for j in range(i,len(feat)):
        if i == j:
            df_temp['%s|%s|mul'%(feat[i],feat[j])] = train[feat[i]]*train[feat[j]]
        else:
            df_temp['%s|%s|add'%(feat[i],feat[j])] = train[feat[i]]+train[feat[j]]
            df_temp['%s|%s|sub'%(feat[i],feat[j])] = train[feat[i]]-train[feat[j]]
            df_temp['%s|%s|mul'%(feat[i],feat[j])] = train[feat[i]]*train[feat[j]]
            df_temp['%s|%s|div'%(feat[i],feat[j])] = train[feat[i]]/train[feat[j]]
            df_temp['%s|%s|div'%(feat[j],feat[i])] = train[feat[j]]/train[feat[i]] ## 这里是逆序相除哦，亲。
    model = LGBMClassifier(
        **params, #n_estimators=1000, learning_rate=0.08, max_depth=7, subsample=0.8, colsample_bytree=0.6, n_jobs=4
    )
    model.fit(df_temp.values, train_y)
    qq = pd.Series(model.feature_importances_, index=df_temp.columns).sort_values()
    print(qq)

In [99]:
# if not os.path.exists("trans_data"):
#     os.makedirs("trans_data")
    
# if os.path.exists('trans_data/train_1000_10.pkl'):
#     train = pickle.load(open('trans_data/train_1000_10.pkl', 'rb'))
#     test = pickle.load(open('trans_data/test_1000_10.pkl', 'rb'))
# else:
importanceThreshold = 50
d={'add':'+', 'sub':'-', 'mul':'*', 'div':'/'}
feat0 = feat.copy()
for i in trange(len(feat)):
    df_temp=train[feat0].copy()
    # for j in range(i+1,len(feat)):
#             print('%s|%s|add'%(feat[i],feat[j]))
    for j in range(i,len(feat)):
        if i == j:
            df_temp['%s|%s|mul'%(feat[i],feat[j])] = train[feat[i]]*train[feat[j]]
        else:
            df_temp['%s|%s|add'%(feat[i],feat[j])] = train[feat[i]]+train[feat[j]]
            df_temp['%s|%s|sub'%(feat[i],feat[j])] = train[feat[i]]-train[feat[j]]
            df_temp['%s|%s|mul'%(feat[i],feat[j])] = train[feat[i]]*train[feat[j]]
            df_temp['%s|%s|div'%(feat[i],feat[j])] = train[feat[i]]/train[feat[j]]
            df_temp['%s|%s|div'%(feat[j],feat[i])] = train[feat[j]]/train[feat[i]] ## 这里是逆序相除哦，亲。
    model = LGBMClassifier(
        **params, #n_estimators=1000, learning_rate=0.08, max_depth=7, subsample=0.8, colsample_bytree=0.6, n_jobs=4
    )
    model.fit(df_temp.values, train_y)
    qq = pd.Series(model.feature_importances_, index=df_temp.columns).sort_values()
    print(qq)
    ## 特殊处理重要程度大于100的特征。卧槽，太帅了。
    for col in set(qq.loc[qq>importanceThreshold].index)-set(feat0):
        f0, f1, f2 = col.split('|')
        train[col] = df_temp[col]
        ## 如果这个特征靠谱，就把这个特征重复做到test数据集里面。这种操作手法太6了。
        test[col] = eval("test['%s'] %s test['%s']"%(f0, d[f2], f1))
    feat0.extend(list(set(qq.loc[qq>importanceThreshold].index)-set(feat0)))
    # pickle.dump(train, open('trans_data/train_1000_10.pkl','wb'))
    # pickle.dump(test, open('trans_data/test_1000_10.pkl','wb'))

train.columns

 11%|█████████▎                                                                          | 1/9 [00:01<00:15,  1.89s/it]

DKLL|GRYJCE|sub           2
DKLL|GRYJCE|add           2
DKLL|GRYJCE|div           3
DKLL|GRYJCE|mul           3
DKLL|DKLL|mul             3
DKLL|DWYJCE|sub           6
GRYJCE|DKLL|div           6
DKLL|DWYJCE|add          10
GRJCJS|DKLL|div          14
DKLL|GRZHDNGJYE|sub      16
DKLL|GRZHSNJZYE|sub      16
DKLL|GRZHSNJZYE|add      16
GRZHSNJZYE|DKLL|div      16
DKLL|GRJCJS|sub          17
DKLL|DKFFE|sub           17
GRZHSNJZYE               20
DKLL|DKYE|div            22
DWYJCE|DKLL|div          23
DKFFE|DKLL|div           24
DKLL|GRZHYE|sub          24
DKLL|DKFFE|mul           26
GRZHYE|DKLL|div          26
DKLL|DWYJCE|mul          27
DKLL|GRZHYE|add          28
DKLL                     28
DKLL|GRJCJS|div          28
DKLL|DKFFE|add           28
DKLL|GRZHDNGJYE|mul      29
DKYE|DKLL|div            29
DKLL|GRZHDNGJYE|add      30
DKLL|GRZHSNJZYE|div      30
GRYJCE                   31
GRZHDNGJYE|DKLL|div      34
DKLL|DKYE|sub            35
DKLL|GRZHYE|div          36
DKLL|DKYE|add       

 22%|██████████████████▋                                                                 | 2/9 [00:03<00:13,  1.88s/it]

GRJCJS|GRYJCE|sub           0
GRJCJS|GRYJCE|add           0
GRJCJS|GRYJCE|div           1
GRJCJS|DWYJCE|add           2
GRJCJS|GRYJCE|mul           3
GRJCJS|DWYJCE|sub           3
GRYJCE|GRJCJS|div           5
GRJCJS|GRJCJS|mul           8
DKFFE|GRJCJS|div           11
GRJCJS|GRZHSNJZYE|sub      14
GRJCJS|DWYJCE|mul          19
GRZHYE|GRJCJS|div          22
GRZHSNJZYE                 23
DKLL|GRJCJS|mul            23
GRJCJS|GRZHSNJZYE|add      24
GRJCJS|GRZHSNJZYE|div      26
GRJCJS|GRZHDNGJYE|add      26
GRZHSNJZYE|GRJCJS|div      28
GRJCJS|DKFFE|div           28
DKLL|GRZHDNGJYE|div        30
DKYE|GRJCJS|div            33
DKLL                       33
GRYJCE                     33
GRJCJS|GRZHYE|div          34
DWYJCE|GRJCJS|div          34
GRJCJS|DKYE|div            34
GRZHYE                     39
GRJCJS|DKFFE|mul           39
GRJCJS|GRZHYE|sub          40
GRJCJS|DKYE|add            40
GRZHDNGJYE|GRJCJS|div      41
DKYE                       41
GRJCJS|GRZHYE|add          41
DKLL|DKYE|

 33%|████████████████████████████                                                        | 3/9 [00:05<00:11,  1.88s/it]

...

 89%|██████████████████████████████████████████████████████████████████████████▋         | 8/9 [00:14<00:01,  1.81s/it]

GRZHSNJZYE|GRZHSNJZYE|mul       6
GRZHSNJZYE|GRYJCE|sub          11
GRZHSNJZYE|GRYJCE|add          24
DKLL                           28
GRZHSNJZYE                     28
DKLL|DKYE|mul                  32
GRZHSNJZYE|GRYJCE|div          35
DKYE                           40
GRYJCE|GRZHSNJZYE|div          42
DKFFE                          42
GRZHYE                         42
GRJCJS|DKFFE|add               44
GRZHSNJZYE|GRYJCE|mul          45
DKLL|GRJCJS|mul                52
GRJCJS|DKFFE|sub               54
DKYE|DKFFE|div                 62
DKFFE|DKYE|div                 64
DKFFE|DKYE|sub                 67
DKLL|GRZHDNGJYE|div            72
DKFFE|GRZHYE|mul               81
GRJCJS|GRZHYE|mul              84
GRYJCE                         85
GRJCJS|GRZHDNGJYE|sub         106
GRJCJS|DWYJCE|div             117
GRZHDNGJYE                    199
GRJCJS                        404
DWYJCE                       1234
dtype: int32


100%|████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:16<00:00,  1.82s/it]

GRYJCE|GRYJCE|mul          14
DKLL                       33
DKLL|GRJCJS|mul            40
DKFFE                      42
DKLL|DKYE|mul              42
DKYE                       45
GRZHYE                     47
GRJCJS|DKFFE|sub           52
DKFFE|DKYE|div             61
GRJCJS|DKFFE|add           68
DKFFE|DKYE|sub             72
DKYE|DKFFE|div             78
GRJCJS|GRZHYE|mul          83
DKLL|GRZHDNGJYE|div        86
GRYJCE                     91
GRZHSNJZYE                 98
DKFFE|GRZHYE|mul          101
GRJCJS|DWYJCE|div         115
GRJCJS|GRZHDNGJYE|sub     127
GRZHDNGJYE                211
GRJCJS                    425
DWYJCE                   1169
dtype: int32


Index(['id', 'XINGBIE', 'CSNY', 'DWJJLX', 'DWSSHY', 'GRJCJS', 'GRZHZT',
       'GRZHYE', 'GRZHSNJZYE', 'GRZHDNGJYE', 'GRYJCE', 'DWYJCE', 'DKFFE',
       'DKYE', 'DKLL', 'label', 'DKLL|GRJCJS|mul', 'DKLL|DKYE|mul',
       'DKLL|GRZHDNGJYE|div', 'GRJCJS|DKFFE|add', 'GRJCJS|GRZHYE|mul',
       'GRJCJS|GRZHDNGJYE|sub', 'GRJCJS|DWYJCE|div', 'GRJCJS|DKFFE|sub',
       'DKFFE|DKYE|sub', 'DKYE|DKFFE|div', 'DKFFE|DKYE|div',
       'DKFFE|GRZHYE|mul'],
      dtype='object')

In [100]:
# # if os.path.exists('trans_data/train_1000_10-1.pkl'):
# #     train = pickle.load(open('trans_data/train_1000_10-1.pkl', 'rb'))
# #     test = pickle.load(open('trans_data/test_1000_10-1.pkl', 'rb'))
# # else:
# #     ### 载入处理好的部分
# #     if os.path.exists('trans_data/train_1000_10.pkl'):
# #         train = pickle.load(open('trans_data/train_1000_10.pkl', 'rb'))
# #         test = pickle.load(open('trans_data/test_1000_10.pkl', 'rb'))
# ### 继续处理
# importanceThreshold = 100
# d={'add':'+', 'sub':'-', 'mul':'*', 'div':'/'}
# feat0 = feat.copy()
# for i in trange(len(feat)):
#     df_temp=train[feat0].copy()
#     print("一开始的时候。。", df_temp.shape)
#     for j in [i]:
#         df_temp['%s|%s|add'%(feat[i],feat[j])] = train[feat[i]]+train[feat[j]]
#         df_temp['%s|%s|mul'%(feat[i],feat[j])] = train[feat[i]]*train[feat[j]]
#     print("后来。。", df_temp.shape)
#     model = LGBMClassifier(
#         **params, #n_estimators=1000, learning_rate=0.08, max_depth=7, subsample=0.8, colsample_bytree=0.6, n_jobs=4
#     )
#     model.fit(df_temp.values, train_y)
#     qq = pd.Series(model.feature_importances_, index=df_temp.columns).sort_values()
#     print(qq)
#     ## 特殊处理重要程度大于某一阈值的特征。卧槽，太帅了。
#     for col in set(qq.loc[qq>importanceThreshold].index)-set(feat0):
#         f0, f1, f2 = col.split('|')
#         train[col] = df_temp[col]
#         ## 如果这个特征靠谱，就把这个特征重复做到test数据集里面。这种操作手法太6了。
#         test[col] = eval("test['%s'] %s test['%s']"%(f0, d[f2], f1))
#     feat0.extend(list(set(qq.loc[qq>importanceThreshold].index)-set(feat0)))
#     # pickle.dump(train, open('trans_data/train_1000_10-1.pkl','wb'))
#     # pickle.dump(test, open('trans_data/test_1000_10-1.pkl','wb'))

# train.columns

In [101]:
def gen_feat(data):
    for col in cat_feat:
        data[col] = data[col].fillna('empty').astype(str)
    for col in data.columns:
        if '年' not in col and '|' not in col and data[col].isna().sum()>0:
            data['%s_na'%col] = data[col].isna().astype(int)
            
    featgen = CatCount(config)
    featgen.transform(data)
    
gen_feat(train)
gen_feat(test)

In [102]:
train.shape

(40000, 32)

In [103]:
train.columns

Index(['id', 'XINGBIE', 'CSNY', 'DWJJLX', 'DWSSHY', 'GRJCJS', 'GRZHZT',
       'GRZHYE', 'GRZHSNJZYE', 'GRZHDNGJYE', 'GRYJCE', 'DWYJCE', 'DKFFE',
       'DKYE', 'DKLL', 'label', 'DKLL|GRJCJS|mul', 'DKLL|DKYE|mul',
       'DKLL|GRZHDNGJYE|div', 'GRJCJS|DKFFE|add', 'GRJCJS|GRZHYE|mul',
       'GRJCJS|GRZHDNGJYE|sub', 'GRJCJS|DWYJCE|div', 'GRJCJS|DKFFE|sub',
       'DKFFE|DKYE|sub', 'DKYE|DKFFE|div', 'DKFFE|DKYE|div',
       'DKFFE|GRZHYE|mul', 'XINGBIE_catcount', 'DWJJLX_catcount',
       'DWSSHY_catcount', 'GRZHZT_catcount'],
      dtype='object')

In [104]:
featgen = CatCountRank(config)
featgen.fit_transform(train, train_y)
featgen.transform(test)

In [105]:
# train[cat_feat].head()

In [106]:
feat0 = list(set(train.columns)-set(train.select_dtypes(object))-set(['label','id'])-set(cat_feat))
remove_col = []
for col in feat0:
    if train[col].nunique() < 2:
        remove_col.append(col)
feat0 = list(set(feat0) - set(remove_col))
feat0

['GRJCJS',
 'GRZHZT_countrank',
 'GRJCJS|DKFFE|add',
 'GRJCJS|GRZHYE|mul',
 'GRZHDNGJYE',
 'GRZHYE',
 'DKLL|GRZHDNGJYE|div',
 'DKFFE|DKYE|sub',
 'GRZHZT_catcount',
 'XINGBIE_catcount',
 'GRZHSNJZYE',
 'DKLL|GRJCJS|mul',
 'DWSSHY_catcount',
 'DWJJLX_countrank',
 'XINGBIE_countrank',
 'DKFFE',
 'CSNY',
 'GRJCJS|GRZHDNGJYE|sub',
 'DKFFE|DKYE|div',
 'DKFFE|GRZHYE|mul',
 'GRJCJS|DWYJCE|div',
 'DKYE|DKFFE|div',
 'DKYE',
 'GRJCJS|DKFFE|sub',
 'DWYJCE',
 'DKLL',
 'DWSSHY_countrank',
 'DWJJLX_catcount',
 'DKLL|DKYE|mul',
 'GRYJCE']

In [107]:
featuresExceptForIDAndLabel = [col for col in train.columns if col not in ["id", "label"] + cat_feat] 
train[featuresExceptForIDAndLabel] = train[featuresExceptForIDAndLabel].apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x)))
test[featuresExceptForIDAndLabel] = test[featuresExceptForIDAndLabel].apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x)))

In [108]:
def tpr_weight_funtion(y_true, y_predict):
    '''
    这是一个通用的计算最终分数的函数。
    '''
    d = pd.DataFrame()
    d['y'] = list(y_true)
    d['prob'] = list(y_predict)
    d = d.sort_values(['prob'], ascending=[0])
    y = d.y
    PosAll = pd.Series(y).value_counts()[1]
    NegAll = pd.Series(y).value_counts()[0]
    pCumsum = d['y'].cumsum()
    nCumsum = np.arange(len(y)) - pCumsum + 1
    pCumsumPer = pCumsum / PosAll
    nCumsumPer = nCumsum / NegAll
    TR1 = pCumsumPer[abs(nCumsumPer-0.001).idxmin()]
    TR2 = pCumsumPer[abs(nCumsumPer-0.005).idxmin()]
    TR3 = pCumsumPer[abs(nCumsumPer-0.01).idxmin()]
    
    return 0.4 * TR1 + 0.3 * TR2 + 0.3 * TR3

In [109]:
def tpr_weight_funtion_lgb(y_true, y_predict):
    '''
    如果是给lgbm用的，参考https://github.com/microsoft/LightGBM/blob/c02917e493c36f3b1e349338f1087fed33126576/examples/python-guide/advanced_example.py#L154
    第一个返回值，是这个函数的可以说是名字或者是标记吧；score就是得到的分数；最后一个就是问，score是越高越好吗。
    '''
#     y_predict = pred
#     y_true = train_data #.get_label()
    
    d = pd.DataFrame()
    d['prob'] = list(y_predict)
    d['y'] = list(y_true)
    d = d.sort_values(['prob'], ascending=[0])
    y = d.y
    PosAll = pd.Series(y).value_counts()[1]
    NegAll = pd.Series(y).value_counts()[0]
    pCumsum = d['y'].cumsum()
    nCumsum = np.arange(len(y)) - pCumsum + 1
    pCumsumPer = pCumsum / PosAll
    nCumsumPer = nCumsum / NegAll
    TR1 = pCumsumPer[abs(nCumsumPer-0.001).idxmin()]
    TR2 = pCumsumPer[abs(nCumsumPer-0.005).idxmin()]
    TR3 = pCumsumPer[abs(nCumsumPer-0.01).idxmin()]
    
    return "Weighted_Score", (0.4 * TR1 + 0.3 * TR2 + 0.3 * TR3), True

######################################################################
kf = KFold(5,True,random_state=1)
prob = np.zeros(len(train))
test_prob = np.zeros(len(test))
test_data = test[feat0].values

cv_scores = []

for idx, (train_index, valid_index) in enumerate(kf.split(train, train_y)):
    print('************************************ {} ************************************'.format(str(idx+1)))

    train_data = train.loc[train_index][feat0].values
    valid_data = train.loc[valid_index][feat0].values
    model = LGBMClassifier(
        **params,
#         n_estimators=10000, 
#         learning_rate=0.08, 
#         num_leaves=15, 
#         subsample=0.8, 
#         colsample_bytree=0.6, 
#         n_jobs=4,
#         categorical_feature = categorical_fea_index, 
    )
    model.fit(
        train_data, train_y.loc[train_index], 
        eval_set=(valid_data, train_y.loc[valid_index]), 
        early_stopping_rounds=200,
        verbose=200,
        eval_metric = tpr_weight_funtion_lgb
    )
    pred = model.predict_proba(valid_data)[:,1]
    prob[valid_index] = pred
#     prob[valid_index] = model.predict_proba(valid_data)[:, 1]
    
    cv_scores.append(tpr_weight_funtion(train_y.iloc[valid_index], pred))
    print(np.mean(cv_scores), cv_scores)
    
    test_prob += model.predict_proba(test_data)[:, 1]/kf.n_splits

************************************ 1 ************************************
Training until validation scores don't improve for 200 rounds
Did not meet early stopping. Best iteration is:
[100]	valid_0's binary_logloss: 0.127953	valid_0's Weighted_Score: 0.447917
0.4479166666666667 [0.4479166666666667]
************************************ 2 ************************************
[LightGBM] [Warning] num_threads is set with n_jobs=24, nthread=28 will be ignored. Current value: num_threads=24
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] lambda_l2 is set=10, reg_lambda=0.0 will be ignored. Current value: lambda_l2=10
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
Training until validation scores don't improve for 200 ro

In [110]:
train['lgb_prob'] = prob
test['lgb_prob'] = test_prob

In [111]:
class score_forCAT(object):
    '''
    https://catboost.ai/docs/concepts/python-usages-examples.html#custom-loss-function-eval-metric
    '''
    def is_max_optimal(self):
        return True

    def evaluate(self, approxes, target, weight):

        ## weight没什么迪奥用。
        assert len(approxes) == 1
        assert len(target) == len(approxes[0])
        
        d = pd.DataFrame()
        d['prob'] = list(approxes[0])
        d['y'] = list(target)
        d = d.sort_values(['prob'], ascending=[0])
        y = d.y
        PosAll = pd.Series(y).value_counts()[1]
        NegAll = pd.Series(y).value_counts()[0]
        pCumsum = d['y'].cumsum()
        nCumsum = np.arange(len(y)) - pCumsum + 1
        pCumsumPer = pCumsum / PosAll
        nCumsumPer = nCumsum / NegAll
        TR1 = pCumsumPer[abs(nCumsumPer-0.001).idxmin()]
        TR2 = pCumsumPer[abs(nCumsumPer-0.005).idxmin()]
        TR3 = pCumsumPer[abs(nCumsumPer-0.01).idxmin()]
        
        score = (0.4 * TR1 + 0.3 * TR2 + 0.3 * TR3)

        return score, -1
    
    def get_final_error(self, error, weight):
        # Returns final value of metric based on error and weight
        return error

kf = KFold(5, True, random_state=1)
prob = np.zeros(len(train))
test_prob = np.zeros(len(test))

feat1=list(set(feat0 + cat_feat + ['lgb_prob']))

cv_scores = []

test_data=test[feat1].values
for idx, (train_index, valid_index) in enumerate(kf.split(train, train_y)):
#     print(list(train_y.iloc[train_index]))
    print('************************************ {} ************************************'.format(str(idx+1)))
    train_data = train.loc[train_index][feat1]
    valid_data = train.loc[valid_index][feat1]
    
    # params_CAT = {
    #     'learning_rate': 0.1, 
    #     'depth': 12, ## 曾经测试过15，但是一旦设置到15，就会奇慢无比
    #     'l2_leaf_reg': 80, #100 #50 # 设为20，能够达到40xx的分数。 
    #     'bootstrap_type': "Bernoulli", ## "Bayesian"比Bernoulli 略强点. 然后我们试试Bernoulli和subsample参数的组合吧。
    #     # "bagging_temperature": 2, ## 这个参数要跟bayesian bootstrap方法组合起来用。
    #     "subsample": 0.7, ## 窝槽，bernoulli+subsample加起来很吊啊。。。
    #     # "sampling_frequency": "PerTree", ## 这个设了没设一样。

    #     "grow_policy": "Depthwise", ## 这个有奇效。应该还有潜力可挖。
    #     "min_data_in_leaf": 2, ## 这个也有效果。默认是1，我估计这样搞就太精确了，容易导致过拟合。所以把区间放大一点效果反倒好。
    #     ## 上面两个，能够达到4236的分数。

    #     ## 如果要用这个参数，不要弄onehot。都交给catboost吧。
    #     "one_hot_max_size": 255, ## 能提高一点点。0.0002吧。

    #     ## 设置的大了，性能可能会下降。原档案警告如上。
    #     "fold_permutation_block": 2, ## 可能有效。可以关注一下。 设为2，达到了4250

    #     ## Balanced跑了两次，最高得到了4286的结果。不太稳定。
    #     "auto_class_weights": "SqrtBalanced",## 得到了4484，升级明显，多测几次，看看是不是偶然 

    #     ## 这个参数酌情不设吧。因为这个的作用应该跟StratifiedKFold一样吧。
    #     "allow_const_label": True,

    #     'od_type': 'Iter', 
    #     'od_wait': 300, 

    #     ## MinEntropy和默认值比较好，其他的比较拉垮。
    #     "feature_border_type": "MinEntropy", 

    #     'random_seed': 11, 
    #     'allow_writing_files': False, 
    #     "task_type": "GPU",
    #     "eval_metric": "AUC",
    # }
    params_CAT = {'learning_rate': 0.05, 'depth': 5, 'l2_leaf_reg': 10, 'bootstrap_type': 'Bernoulli',
                      'od_type': 'Iter', 'od_wait': 50, 'random_seed': 11, 'allow_writing_files': False, 
#                      "task_type": "GPU",
                     }
    model = CatBoostClassifier(
        **params_CAT, 
        #iterations=10000, learning_rate=0.08, depth=7, 
        cat_features=cat_feat, 
        # eval_metric= score_forCAT(), 
        use_best_model= True
    )
    model.fit(train_data, train_y.loc[train_index], 
              eval_set=(valid_data, train_y.loc[valid_index]), early_stopping_rounds=50, 
             verbose=500
#              eval_metric = score_forCAT()
             )
    pred = model.predict_proba(valid_data)[:,1]
    prob[valid_index] = pred
    test_prob += model.predict_proba(test_data)[:,1]/5
    
    cv_scores.append(tpr_weight_funtion(train_y.iloc[valid_index], pred))
    print(np.mean(cv_scores), cv_scores)



************************************ 1 ************************************
0:	learn: 0.6210384	test: 0.6214307	best: 0.6214307 (0)	total: 53.1ms	remaining: 53.1s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.1261628194
bestIteration = 399

Shrink model to first 400 iterations.
0.4447916666666667 [0.4447916666666667]
************************************ 2 ************************************
0:	learn: 0.6212729	test: 0.6214386	best: 0.6214386 (0)	total: 40.1ms	remaining: 40.1s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.1244810609
bestIteration = 238

Shrink model to first 239 iterations.
0.43873335577498507 [0.4447916666666667, 0.43267504488330344]
************************************ 3 ************************************
0:	learn: 0.6213045	test: 0.6215205	best: 0.6215205 (0)	total: 44.9ms	remaining: 44.8s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.1274414094
bestIteration = 339

Shrink model to first 340 iterati

In [ ]:
test['label'] = test_prob
if not os.path.exists("output"):
    os.makedirs("output")
test[['id', 'label']].to_csv('submissionResults/4.1-FENew-4537.csv', index=False)